In [1]:
%cd ../..

/home/dmoreno/ATAT_ZTF/elasticc_one


In [48]:
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import glob

In [49]:
dict_cols = {
    'oid': 'oid',
    'time': 'mjd',
    'flux': 'flux_diff_ujy',
    'flux_err': 'sigma_flux_diff_ujy',
    'detected': 'detected',
    'band': 'fid',
    'class': 'alerceclass'
}

In [50]:
# Astronomical objects and their labels
df_objid_label = pd.read_parquet('./data/datasets/ZTF_ff/processed/data_231206/objects.parquet')
df_objid_label = df_objid_label.reset_index()
df_objid_label = df_objid_label[[dict_cols['oid'], dict_cols['class']]]
df_objid_label.head()

,oid,alerceclass
0,ZTF21acasmne,CV/Nova
1,ZTF19aaonuwr,LPV
2,ZTF17aabhkmn,YSO
3,ZTF18aatxhtl,QSO
4,ZTF18aaadfyl,RSCVn


In [51]:
path_chunks = sorted(glob.glob('./data/datasets/ZTF_ff/processed/data_231206/lightcurves_batch_*'))

df_lcs = []
for i, path_chunk in enumerate(path_chunks):
    print('Processing chunk {}'.format(i))
    df_chunk = pd.read_parquet('{}'.format(path_chunk))
    df_chunk = df_chunk[df_chunk[dict_cols['oid']].isin(df_objid_label[dict_cols['oid']].values)]
    df_chunk = df_chunk.groupby(dict_cols['oid']).apply(lambda x: x.sort_values(dict_cols['time'])).reset_index(drop=True)
    df_chunk_filtered = df_chunk[list(set(dict_cols.values()) - {dict_cols['class']})]
    df_lcs.append(df_chunk_filtered)

df_lcs = pd.concat(df_lcs)

Processing chunk 0
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7
Processing chunk 8
Processing chunk 9
Processing chunk 10
Processing chunk 11
Processing chunk 12
Processing chunk 13
Processing chunk 14
Processing chunk 15
Processing chunk 16
Processing chunk 17
Processing chunk 18
Processing chunk 19
Processing chunk 20
Processing chunk 21
Processing chunk 22
Processing chunk 23
Processing chunk 24
Processing chunk 25
Processing chunk 26
Processing chunk 27
Processing chunk 28
Processing chunk 29
Processing chunk 30
Processing chunk 31
Processing chunk 32
Processing chunk 33
Processing chunk 34
Processing chunk 35
Processing chunk 36
Processing chunk 37
Processing chunk 38
Processing chunk 39
Processing chunk 40
Processing chunk 41
Processing chunk 42
Processing chunk 43
Processing chunk 44
Processing chunk 45
Processing chunk 46
Processing chunk 47
Processing chunk 48
Processing chunk 49
Processing

In [52]:
df_lcs_windows = df_lcs.copy()
df_lcs_windows['window_id'] = df_lcs_windows.groupby('oid').cumcount() // 200
df_lcs_windows = df_lcs_windows.groupby(["oid", "window_id"]).agg(lambda x: list(x))
df_lcs_windows

detected   
oid          window_id                                                      
ZTF17aaaaabj 0          [True, True, True, False, True, True, True, Tr...  \
             1          [True, True, True, True, True, True, True, Tru...   
             2          [True, True, True, True, True, True, True, Tru...   
             3          [False, False, False, True, True, True, True, ...   
             4          [True, True, True, True, True, True, True, Tru...   
...                                                                   ...   
ZTF23aboxuyi 0          [False, False, False, False, False, False, Fal...   
             1          [False, False, False, False, False, False, Tru...   
             2          [False, False, False, False, False, False, Fal...   
ZTF23abpbdyh 0          [False, False, False, False, False, False, Fal...   
ZTF23abpbuha 0                      [True, False, True, True, True, True]   

                                                                      mjd   
oid          window_id                                                      
ZTF17aaaaabj 0          [58206.16430559987, 58231.14488429995, 58234.1...  \
             1          [58491.22325230017, 58493.221169000026, 58493....   
             2          [58806.42067130003, 58806.421134300064, 58806....   
             3          [59043.470949099865, 59043.47599539999, 59043....   
             4          [59294.158067100216, 59294.19618060021, 59296....   
...                                                                   ...   
ZTF23aboxuyi 0          [58872.165138899814, 58875.09256940009, 58875....   
             1          [59418.46142360009, 59422.43584490009, 59426.4...   
             2          [59803.41695600003, 59803.45730319992, 59808.4...   
ZTF23abpbdyh 0          [60236.48218750022, 60238.51758099999, 60242.4...   
ZTF23abpbuha 0          [60248.48346060002, 60248.50972220022, 60251.4...   

                                                      sigma_flux_diff_ujy   
oid          window_id                                                      
ZTF17aaaaabj 0          [8.440242655407886, 9.70802999032586, 20.01827...  \
             1          [4.335927383739846, 2.337136437970241, 2.61231...   
             2          [3.3180606229102576, 3.4208377390652815, 3.416...   
             3          [4.021680458844159, 4.18601374614786, 4.365849...   
             4          [7.990314418768771, 8.825867966180457, 10.4053...   
...                                                                   ...   
ZTF23aboxuyi 0          [4.789168652528449, 6.6113980531103085, 4.0210...   
             1          [10.31127100662421, 7.095175466434133, 4.76234...   
             2          [8.438503543741724, 9.441484804282839, 5.70083...   
ZTF23abpbdyh 0          [5.791383450140314, 3.544662523295157, 2.92826...   
ZTF23abpbuha 0          [13.62036221217015, 14.48718316429837, 11.4438...   

                                                            flux_diff_ujy   
oid          window_id                                                      
ZTF17aaaaabj 0          [-249.57181969652103, 52.815739767359396, -253...  \
             1          [318.88202860232195, -230.68964424237782, -148...   
             2          [-343.69137256798797, -341.49161921565184, -35...   
             3          [19.587287226180354, 4.68433612312114, 0.74238...   
             4          [196.79623824291224, 301.52910950582645, -288....   
...                                                                   ...   
ZTF23aboxuyi 0          [1.1547001816474889, -0.84922994538376, 0.5157...   
             1          [22.27578010361939, 27.833914276248418, 15.723...   
             2          [13.20777222894057, 17.15569121649437, 12.1412...   
ZTF23abpbdyh 0          [6.666708298405723, 3.4420276517688113, 3.3954...   
ZTF23abpbuha 0          [169.48633315872158, 32.28439269439875, 157.16...   

                                                         

In [53]:
windows_merged_df = pd.merge(df_lcs_windows.reset_index(), df_objid_label, on='oid', how='outer')
windows_merged_df

,oid,window_id,detected,mjd,sigma_flux_diff_ujy,flux_diff_ujy,fid,alerceclass
0,ZTF17aaaaabj,0,"[True, True, True, False, True, True, True, Tr...","[58206.16430559987, 58231.14488429995, 58234.1...","[8.440242655407886, 9.70802999032586, 20.01827...","[-249.57181969652103, 52.815739767359396, -253...","[1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, ...",RRLc
1,ZTF17aaaaabj,1,"[True, True, True, True, True, True, True, Tru...","[58491.22325230017, 58493.221169000026, 58493....","[4.335927383739846, 2.337136437970241, 2.61231...","[318.88202860232195, -230.68964424237782, -148...","[2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, ...",RRLc
2,ZTF17aaaaabj,2,"[True, True, True, True, True, True, True, Tru...","[58806.42067130003, 58806.421134300064, 58806....","[3.3180606229102576, 3.4208377390652815, 3.416...","[-343.69137256798797, -341.49161921565184, -35...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",RRLc
3,ZTF17aaaaabj,3,"[False, False, False, True, True, True, True, ...","[59043.470949099865, 59043.47599539999, 59043....","[4.021680458844159, 4.18601374614786, 4.365849...","[19.587287226180354, 4.68433612312114, 0.74238...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",RRLc
4,ZTF17aaaaabj,4,"[True, True, True, True, True, True, True, Tru...","[59294.158067100216, 59294.19618060021, 59296....","[7.990314418768771, 8.825867966180457, 10.4053...","[196.79623824291224, 301.52910950582645, -288....","[1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",RRLc
...,...,...,...,...,...,...,...,...
253012,ZTF23aboxuyi,0,"[False, False, False, False, False, False, Fal...","[58872.165138899814, 58875.09256940009, 58875....","[4.789168652528449, 6.6113980531103085, 4.0210...","[1.1547001816474889, -0.84922994538376, 0.5157...","[1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, ...",QSO
253013,ZTF23aboxuyi,1,"[False, False, False, False, False, False, Tru...","[59418.46142360009, 59422.43584490009, 59426.4...","[10.31127100662421, 7.095175466434133, 4.76234...","[22.27578010361939, 27.833914276248418, 15.723...","[1, 2, 2, 1, 1, 3, 2, 1, 2, 1, 3, 2, 3, 2, 1, ...",QSO
253014,ZTF23aboxuyi,2,"[False, False, False, False, False, False, Fal...","[59803.41695600003, 59803.45730319992, 59808.4...","[8.438503543741724, 9.441484804282839, 5.70083...","[13.20777222894057, 17.15569121649437, 12.1412...","[2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, ...",QSO
253015,ZTF23abpbdyh,0,"[False, False, False, False, False, False, Fal...","[60236.48218750022, 60238.51758099999, 60242.4...","[5.791383450140314, 3.544662523295157, 2.92826...","[6.666708298405723, 3.4420276517688113, 3.3954...","[2, 2, 1, 2, 1, 2, 1, 1, 1, 2]",CV/Nova


In [54]:
final_windows_merged_df = windows_merged_df.set_index(['oid', 'window_id'])
final_windows_merged_df

detected   
oid          window_id                                                      
ZTF17aaaaabj 0          [True, True, True, False, True, True, True, Tr...  \
             1          [True, True, True, True, True, True, True, Tru...   
             2          [True, True, True, True, True, True, True, Tru...   
             3          [False, False, False, True, True, True, True, ...   
             4          [True, True, True, True, True, True, True, Tru...   
...                                                                   ...   
ZTF23aboxuyi 0          [False, False, False, False, False, False, Fal...   
             1          [False, False, False, False, False, False, Tru...   
             2          [False, False, False, False, False, False, Fal...   
ZTF23abpbdyh 0          [False, False, False, False, False, False, Fal...   
ZTF23abpbuha 0                      [True, False, True, True, True, True]   

                                                                      mjd   
oid          window_id                                                      
ZTF17aaaaabj 0          [58206.16430559987, 58231.14488429995, 58234.1...  \
             1          [58491.22325230017, 58493.221169000026, 58493....   
             2          [58806.42067130003, 58806.421134300064, 58806....   
             3          [59043.470949099865, 59043.47599539999, 59043....   
             4          [59294.158067100216, 59294.19618060021, 59296....   
...                                                                   ...   
ZTF23aboxuyi 0          [58872.165138899814, 58875.09256940009, 58875....   
             1          [59418.46142360009, 59422.43584490009, 59426.4...   
             2          [59803.41695600003, 59803.45730319992, 59808.4...   
ZTF23abpbdyh 0          [60236.48218750022, 60238.51758099999, 60242.4...   
ZTF23abpbuha 0          [60248.48346060002, 60248.50972220022, 60251.4...   

                                                      sigma_flux_diff_ujy   
oid          window_id                                                      
ZTF17aaaaabj 0          [8.440242655407886, 9.70802999032586, 20.01827...  \
             1          [4.335927383739846, 2.337136437970241, 2.61231...   
             2          [3.3180606229102576, 3.4208377390652815, 3.416...   
             3          [4.021680458844159, 4.18601374614786, 4.365849...   
             4          [7.990314418768771, 8.825867966180457, 10.4053...   
...                                                                   ...   
ZTF23aboxuyi 0          [4.789168652528449, 6.6113980531103085, 4.0210...   
             1          [10.31127100662421, 7.095175466434133, 4.76234...   
             2          [8.438503543741724, 9.441484804282839, 5.70083...   
ZTF23abpbdyh 0          [5.791383450140314, 3.544662523295157, 2.92826...   
ZTF23abpbuha 0          [13.62036221217015, 14.48718316429837, 11.4438...   

                                                            flux_diff_ujy   
oid          window_id                                                      
ZTF17aaaaabj 0          [-249.57181969652103, 52.815739767359396, -253...  \
             1          [318.88202860232195, -230.68964424237782, -148...   
             2          [-343.69137256798797, -341.49161921565184, -35...   
             3          [19.587287226180354, 4.68433612312114, 0.74238...   
             4          [196.79623824291224, 301.52910950582645, -288....   
...                                                                   ...   
ZTF23aboxuyi 0          [1.1547001816474889, -0.84922994538376, 0.5157...   
             1          [22.27578010361939, 27.833914276248418, 15.723...   
             2          [13.20777222894057, 17.15569121649437, 12.1412...   
ZTF23abpbdyh 0          [6.666708298405723, 3.4420276517688113, 3.3954...   
ZTF23abpbuha 0          [169.48633315872158, 32.28439269439875, 157.16...   

                                                         

In [55]:
final_windows_merged_df.groupby(['alerceclass']).count()[['mjd']].sort_values(by='mjd', ascending=False)

,mjd
alerceclass,
EB/EW,22932
RRLab,22005
EA,21830
QSO,21384
RRLc,21038
LPV,20800
AGN,19880
CEP,17584
YSO,16492


In [56]:
df_objid_label.groupby(['alerceclass']).count()[['oid']].sort_values(by='oid', ascending=False)

,oid
alerceclass,
LPV,3003
RRLc,3000
EA,3000
EB/EW,3000
RRLab,2999
QSO,2999
YSO,2998
AGN,2998
CEP,2998


In [ ]:
def create_windows(lc, max_obs):
    windows = []
    for inicio in range(0, len(lc), max_obs):
        fin = inicio + max_obs
        window = lc.iloc[inicio:fin]
        windows.append(window)
    return windows

def get_windows(df_chunk_filtered, dict_cols):
    list_df_windows = \
        df_chunk_filtered.groupby(dict_cols['oid']).apply(lambda x: create_windows(x, 200))
    
    dict_num_windows = dict()
    lcs_windows = []
    for _, df_windows in list_df_windows.items():
        for j, df_window in enumerate(df_windows):
            dict_num_windows[df_window[dict_cols['oid']].iloc[0]] = j
            df_window[dict_cols['oid']] = df_window[dict_cols['oid']].iloc[0] + '_{}'.format(j)
            lcs_windows.append(df_window)
        
    df_chunk_filtered = pd.concat(lcs_windows)
    return df_chunk_filtered, dict_num_windows


df_a = get_windows(df_lcs, dict_cols)
df_a

(        sigma_flux_diff_ujy  detected  flux_diff_ujy  fid           mjd   
 0                  8.440243      True    -249.571820    1  58206.164306  \
 1                  9.708030      True      52.815740    2  58231.144884   
 2                 20.018277      True    -253.767498    1  58234.141609   
 3                  8.486230     False      41.734686    2  58303.480729   
 4                  9.187626      True    -269.465062    1  58304.481157   
 ...                     ...       ...            ...  ...           ...   
 577238            14.487183     False      32.284393    1  60248.509722   
 577239            11.443844      True     157.167240    2  60251.481586   
 577240             6.817979      True      47.698061    1  60251.506343   
 577241             5.679314      True     184.476503    2  60253.497951   
 577242             5.894150      True     182.685772    2  60254.503461   
 
                    oid  
 0       ZTF17aaaaabj_0  
 1       ZTF17aaaaabj_0  
 2      

In [ ]:
df_a[0]

,sigma_flux_diff_ujy,detected,flux_diff_ujy,fid,mjd,oid
0,8.440243,True,-249.571820,1,58206.164306,ZTF17aaaaabj_0
1,9.708030,True,52.815740,2,58231.144884,ZTF17aaaaabj_0
2,20.018277,True,-253.767498,1,58234.141609,ZTF17aaaaabj_0
3,8.486230,False,41.734686,2,58303.480729,ZTF17aaaaabj_0
4,9.187626,True,-269.465062,1,58304.481157,ZTF17aaaaabj_0
...,...,...,...,...,...,...
577238,14.487183,False,32.284393,1,60248.509722,ZTF23abpbuha_0
577239,11.443844,True,157.167240,2,60251.481586,ZTF23abpbuha_0
577240,6.817979,True,47.698061,1,60251.506343,ZTF23abpbuha_0
577241,5.679314,True,184.476503,2,60253.497951,ZTF23abpbuha_0


In [ ]:
df_a[1]

{'ZTF17aaaaabj': 6,
 'ZTF17aaaaaej': 6,
 'ZTF17aaaaafz': 6,
 'ZTF17aaaaajg': 7,
 'ZTF17aaaaavq': 10,
 'ZTF17aaaaaya': 8,
 'ZTF17aaaabbo': 6,
 'ZTF17aaaabhx': 6,
 'ZTF17aaaacnc': 2,
 'ZTF17aaaactd': 3,
 'ZTF17aaaacth': 3,
 'ZTF17aaaadbq': 5,
 'ZTF17aaaadmx': 5,
 'ZTF17aaaaepx': 5,
 'ZTF17aaaafbz': 8,
 'ZTF17aaaafcd': 10,
 'ZTF17aaaafdm': 9,
 'ZTF17aaaagai': 5,
 'ZTF17aaaaghy': 5,
 'ZTF17aaaagvx': 5,
 'ZTF17aaaagym': 5,
 'ZTF17aaaagyq': 5,
 'ZTF17aaaagyx': 5,
 'ZTF17aaaahsl': 6,
 'ZTF17aaaajey': 11,
 'ZTF17aaaajez': 11,
 'ZTF17aaaajgm': 10,
 'ZTF17aaaajgo': 10,
 'ZTF17aaaajhl': 10,
 'ZTF17aaaajhn': 10,
 'ZTF17aaaajhz': 11,
 'ZTF17aaaajib': 11,
 'ZTF17aaaajim': 10,
 'ZTF17aaaajio': 10,
 'ZTF17aaaajiy': 10,
 'ZTF17aaaajji': 10,
 'ZTF17aaaajka': 8,
 'ZTF17aaaajor': 10,
 'ZTF17aaaajow': 11,
 'ZTF17aaaajoz': 11,
 'ZTF17aaaajqk': 8,
 'ZTF17aaaajqs': 9,
 'ZTF17aaaajqz': 9,
 'ZTF17aaaajra': 9,
 'ZTF17aaaakgn': 7,
 'ZTF17aaaakif': 6,
 'ZTF17aaaakmu': 6,
 'ZTF17aaaamne': 6,
 'ZTF17aaaaniv': 5,
 'Z

In [ ]:
final_windows_merged_df

sigma_flux_diff_ujy   
oid          window_id                                                      
ZTF17aaaaabj 0          [8.440242655407886, 9.70802999032586, 20.01827...  \
             1          [4.335927383739846, 2.337136437970241, 2.61231...   
             2          [3.3180606229102576, 3.4208377390652815, 3.416...   
             3          [4.021680458844159, 4.18601374614786, 4.365849...   
             4          [7.990314418768771, 8.825867966180457, 10.4053...   
...                                                                   ...   
ZTF23aboxuyi 0          [4.789168652528449, 6.6113980531103085, 4.0210...   
             1          [10.31127100662421, 7.095175466434133, 4.76234...   
             2          [8.438503543741724, 9.441484804282839, 5.70083...   
ZTF23abpbdyh 0          [5.791383450140314, 3.544662523295157, 2.92826...   
ZTF23abpbuha 0          [13.62036221217015, 14.48718316429837, 11.4438...   

                                                                 detected   
oid          window_id                                                      
ZTF17aaaaabj 0          [True, True, True, False, True, True, True, Tr...  \
             1          [True, True, True, True, True, True, True, Tru...   
             2          [True, True, True, True, True, True, True, Tru...   
             3          [False, False, False, True, True, True, True, ...   
             4          [True, True, True, True, True, True, True, Tru...   
...                                                                   ...   
ZTF23aboxuyi 0          [False, False, False, False, False, False, Fal...   
             1          [False, False, False, False, False, False, Tru...   
             2          [False, False, False, False, False, False, Fal...   
ZTF23abpbdyh 0          [False, False, False, False, False, False, Fal...   
ZTF23abpbuha 0                      [True, False, True, True, True, True]   

                                                            flux_diff_ujy   
oid          window_id                                                      
ZTF17aaaaabj 0          [-249.57181969652103, 52.815739767359396, -253...  \
             1          [318.88202860232195, -230.68964424237782, -148...   
             2          [-343.69137256798797, -341.49161921565184, -35...   
             3          [19.587287226180354, 4.68433612312114, 0.74238...   
             4          [196.79623824291224, 301.52910950582645, -288....   
...                                                                   ...   
ZTF23aboxuyi 0          [1.1547001816474889, -0.84922994538376, 0.5157...   
             1          [22.27578010361939, 27.833914276248418, 15.723...   
             2          [13.20777222894057, 17.15569121649437, 12.1412...   
ZTF23abpbdyh 0          [6.666708298405723, 3.4420276517688113, 3.3954...   
ZTF23abpbuha 0          [169.48633315872158, 32.28439269439875, 157.16...   

                                                                      fid   
oid          window_id                                                      
ZTF17aaaaabj 0          [1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, ...  \
             1          [2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, ...   
             2          [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...   
             3          [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...   
             4          [1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...   
...                                                                   ...   
ZTF23aboxuyi 0          [1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, ...   
             1          [1, 2, 2, 1, 1, 3, 2, 1, 2, 1, 3, 2, 3, 2, 1, ...   
             2          [2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, ...   
ZTF23abpbdyh 0                             [2, 2, 1, 2, 1, 2, 1, 1, 1, 2]   
ZTF23abpbuha 0                                         [2, 1, 2, 1, 2, 2]   

                                              

In [57]:
def open_partitions(partitions, dict_cols):
    mapping_to_int = {key: idx for idx, key in enumerate(partitions[dict_cols['class']].unique())}

    def apply_mapping(label_str):
        return mapping_to_int[label_str]

    partitions['label_int'] = partitions.apply(lambda x: apply_mapping(x[dict_cols['class']]), axis=1)
    return partitions, mapping_to_int

df_partitions = pd.read_parquet('./data/datasets/ZTF_ff/partitions/v3/partitions.parquet')
df_partitions

,oid,alerceclass,partition
index,,,
23577,ZTF18adldhip,AGN,test
32093,ZTF19aasbgeb,QSO,test
17103,ZTF18aayfbqd,EA,test
6117,ZTF19ablyzbl,QSO,test
33717,ZTF18actabfv,YSO,test
...,...,...,...
41704,ZTF19acokmuo,AGN,validation_4
41708,ZTF18abxoxih,Blazar,validation_4
41716,ZTF18adbhpsf,Blazar,validation_4


In [58]:
pp = final_windows_merged_df.reset_index()
pp.head()

,oid,window_id,detected,mjd,sigma_flux_diff_ujy,flux_diff_ujy,fid,alerceclass
0,ZTF17aaaaabj,0,"[True, True, True, False, True, True, True, Tr...","[58206.16430559987, 58231.14488429995, 58234.1...","[8.440242655407886, 9.70802999032586, 20.01827...","[-249.57181969652103, 52.815739767359396, -253...","[1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, ...",RRLc
1,ZTF17aaaaabj,1,"[True, True, True, True, True, True, True, Tru...","[58491.22325230017, 58493.221169000026, 58493....","[4.335927383739846, 2.337136437970241, 2.61231...","[318.88202860232195, -230.68964424237782, -148...","[2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, ...",RRLc
2,ZTF17aaaaabj,2,"[True, True, True, True, True, True, True, Tru...","[58806.42067130003, 58806.421134300064, 58806....","[3.3180606229102576, 3.4208377390652815, 3.416...","[-343.69137256798797, -341.49161921565184, -35...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",RRLc
3,ZTF17aaaaabj,3,"[False, False, False, True, True, True, True, ...","[59043.470949099865, 59043.47599539999, 59043....","[4.021680458844159, 4.18601374614786, 4.365849...","[19.587287226180354, 4.68433612312114, 0.74238...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",RRLc
4,ZTF17aaaaabj,4,"[True, True, True, True, True, True, True, Tru...","[59294.158067100216, 59294.19618060021, 59296....","[7.990314418768771, 8.825867966180457, 10.4053...","[196.79623824291224, 301.52910950582645, -288....","[1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",RRLc


In [ ]:
name_cols = ['oid', 'window_id', 'alerceclass', 'partition']

test_partition = df_partitions[df_partitions['partition'] == 'test']

partitions = []
aa = pd.merge(test_partition, pp.drop(['alerceclass'], axis=1), on='oid', how='outer').dropna()[name_cols]
partitions.append(aa)
partitions[0]

,oid,window_id,alerceclass,partition
0,ZTF18adldhip,0,AGN,test
1,ZTF18adldhip,1,AGN,test
2,ZTF18adldhip,2,AGN,test
3,ZTF18adldhip,3,AGN,test
4,ZTF19aasbgeb,0,QSO,test
...,...,...,...,...
50496,ZTF17aabpjme,6,RRLab,test
50497,ZTF17aabpjme,7,RRLab,test
50498,ZTF18acrwlnr,0,YSO,test
50499,ZTF18acrwlnr,1,YSO,test


In [ ]:
for fold in range(5):
    print('We are expanding the number of windows per light curve in fold {}...'.format(fold))
    this_partitions = df_partitions[(df_partitions['partition'] == 'training_%d' %  fold) | \
                                    (df_partitions['partition'] == 'validation_%d' % fold)]
    
    aa = pd.merge(this_partitions, pp.drop(['alerceclass'], axis=1), on='oid', how='outer').dropna()[name_cols]

    partitions.append(aa)

We are expanding the number of windows per light curve in fold 0...
We are expanding the number of windows per light curve in fold 1...
We are expanding the number of windows per light curve in fold 2...
We are expanding the number of windows per light curve in fold 3...
We are expanding the number of windows per light curve in fold 4...


In [ ]:
aa

,oid,window_id,alerceclass,partition
0,ZTF21acasmne,0,CV/Nova,training_4
1,ZTF21acasmne,1,CV/Nova,training_4
2,ZTF21acasmne,2,CV/Nova,training_4
3,ZTF21acasmne,3,CV/Nova,training_4
4,ZTF17aabhkmn,0,YSO,training_4
...,...,...,...,...
202304,ZTF18adbhpsf,2,Blazar,validation_4
202305,ZTF18adbhpsf,3,Blazar,validation_4
202306,ZTF21aamkwra,0,Periodic-Other,validation_4
202307,ZTF21aamkwra,1,Periodic-Other,validation_4


In [ ]:
aa.oid.nunique()

33382

In [ ]:
def select_random_window_id(group):
    return group.sample(n=1)

# Agrupamos por 'oid' y aplicamos la función
random_window_ids = aa.groupby('oid').apply(select_random_window_id).reset_index(drop=True)
random_window_ids

,oid,window_id,alerceclass,partition
0,ZTF17aaaaabj,2,RRLc,validation_4
1,ZTF17aaaaaej,3,RRLc,training_4
2,ZTF17aaaaajg,0,RRLc,validation_4
3,ZTF17aaaaavq,10,LPV,training_4
4,ZTF17aaaaaya,1,RRLab,validation_4
...,...,...,...,...
33377,ZTF23abobwsd,0,SNII,training_4
33378,ZTF23abofpay,0,SNIbc,training_4
33379,ZTF23aboxuyi,1,QSO,training_4
33380,ZTF23abpbdyh,0,CV/Nova,training_4


In [ ]:
aa

,oid,window_id,alerceclass,partition
0,ZTF21acasmne,0,CV/Nova,training_4
1,ZTF21acasmne,1,CV/Nova,training_4
2,ZTF21acasmne,2,CV/Nova,training_4
3,ZTF21acasmne,3,CV/Nova,training_4
4,ZTF17aabhkmn,0,YSO,training_4
...,...,...,...,...
202304,ZTF18adbhpsf,2,Blazar,validation_4
202305,ZTF18adbhpsf,3,Blazar,validation_4
202306,ZTF21aamkwra,0,Periodic-Other,validation_4
202307,ZTF21aamkwra,1,Periodic-Other,validation_4


In [ ]:
partitions[5]

,oid,window_id,alerceclass,partition
0,ZTF21acasmne,0,CV/Nova,training_4
1,ZTF21acasmne,1,CV/Nova,training_4
2,ZTF21acasmne,2,CV/Nova,training_4
3,ZTF21acasmne,3,CV/Nova,training_4
4,ZTF17aabhkmn,0,YSO,training_4
...,...,...,...,...
202304,ZTF18adbhpsf,2,Blazar,validation_4
202305,ZTF18adbhpsf,3,Blazar,validation_4
202306,ZTF21aamkwra,0,Periodic-Other,validation_4
202307,ZTF21aamkwra,1,Periodic-Other,validation_4


In [ ]:
filtered_0 = pd.merge(partitions[1], random_window_ids[['oid', 'window_id']], on=['oid', 'window_id'])
filtered_0

,oid,window_id,alerceclass,partition
0,ZTF21acasmne,0,CV/Nova,training_0
1,ZTF17aabhkmn,3,YSO,training_0
2,ZTF18aatxhtl,5,QSO,training_0
3,ZTF18aaadfyl,2,RSCVn,training_0
4,ZTF19aapocaj,0,QSO,training_0
...,...,...,...,...
33377,ZTF18aazngol,1,LPV,validation_0
33378,ZTF18aaxjkjm,6,EB/EW,validation_0
33379,ZTF18adruaae,3,YSO,validation_0
33380,ZTF18acacrgm,5,EB/EW,validation_0


In [ ]:
pp.drop(['alerceclass'], axis=1)

,oid,window_id,detected,mjd,fid,flux_diff_ujy,sigma_flux_diff_ujy
0,ZTF17aaaaabj,0,"[True, True, True, False, True, True, True, Tr...","[58206.16430559987, 58231.14488429995, 58234.1...","[1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, ...","[-249.57181969652103, 52.815739767359396, -253...","[8.440242655407886, 9.70802999032586, 20.01827..."
1,ZTF17aaaaabj,1,"[True, True, True, True, True, True, True, Tru...","[58491.22325230017, 58493.221169000026, 58493....","[2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, ...","[318.88202860232195, -230.68964424237782, -148...","[4.335927383739846, 2.337136437970241, 2.61231..."
2,ZTF17aaaaabj,2,"[True, True, True, True, True, True, True, Tru...","[58806.42067130003, 58806.421134300064, 58806....","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[-343.69137256798797, -341.49161921565184, -35...","[3.3180606229102576, 3.4208377390652815, 3.416..."
3,ZTF17aaaaabj,3,"[False, False, False, True, True, True, True, ...","[59043.470949099865, 59043.47599539999, 59043....","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[19.587287226180354, 4.68433612312114, 0.74238...","[4.021680458844159, 4.18601374614786, 4.365849..."
4,ZTF17aaaaabj,4,"[True, True, True, True, True, True, True, Tru...","[59294.158067100216, 59294.19618060021, 59296....","[1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[196.79623824291224, 301.52910950582645, -288....","[7.990314418768771, 8.825867966180457, 10.4053..."
...,...,...,...,...,...,...,...
253012,ZTF23aboxuyi,0,"[False, False, False, False, False, False, Fal...","[58872.165138899814, 58875.09256940009, 58875....","[1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, ...","[1.1547001816474889, -0.84922994538376, 0.5157...","[4.789168652528449, 6.6113980531103085, 4.0210..."
253013,ZTF23aboxuyi,1,"[False, False, False, False, False, False, Tru...","[59418.46142360009, 59422.43584490009, 59426.4...","[1, 2, 2, 1, 1, 3, 2, 1, 2, 1, 3, 2, 3, 2, 1, ...","[22.27578010361939, 27.833914276248418, 15.723...","[10.31127100662421, 7.095175466434133, 4.76234..."
253014,ZTF23aboxuyi,2,"[False, False, False, False, False, False, Fal...","[59803.41695600003, 59803.45730319992, 59808.4...","[2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, ...","[13.20777222894057, 17.15569121649437, 12.1412...","[8.438503543741724, 9.441484804282839, 5.70083..."
253015,ZTF23abpbdyh,0,"[False, False, False, False, False, False, Fal...","[60236.48218750022, 60238.51758099999, 60242.4...","[2, 2, 1, 2, 1, 2, 1, 1, 1, 2]","[6.666708298405723, 3.4420276517688113, 3.3954...","[5.791383450140314, 3.544662523295157, 2.92826..."


In [ ]:
df = pd.concat(partitions)
df

,oid,window_id,alerceclass,partition
0,ZTF18adldhip,0,AGN,test
1,ZTF18adldhip,1,AGN,test
2,ZTF18adldhip,2,AGN,test
3,ZTF18adldhip,3,AGN,test
4,ZTF19aasbgeb,0,QSO,test
...,...,...,...,...
202304,ZTF18adbhpsf,2,Blazar,validation_4
202305,ZTF18adbhpsf,3,Blazar,validation_4
202306,ZTF21aamkwra,0,Periodic-Other,validation_4
202307,ZTF21aamkwra,1,Periodic-Other,validation_4


In [ ]:
df['oid'] = df['oid'].astype(str) + '_' + df['window_id'].astype(str)
df = df.drop(['window_id'], axis=1)

In [ ]:
df

,oid,alerceclass,partition
0,ZTF18adldhip_0,AGN,test
1,ZTF18adldhip_1,AGN,test
2,ZTF18adldhip_2,AGN,test
3,ZTF18adldhip_3,AGN,test
4,ZTF19aasbgeb_0,QSO,test
...,...,...,...
202304,ZTF18adbhpsf_2,Blazar,validation_4
202305,ZTF18adbhpsf_3,Blazar,validation_4
202306,ZTF21aamkwra_0,Periodic-Other,validation_4
202307,ZTF21aamkwra_1,Periodic-Other,validation_4


In [ ]:
import copy

def expands_ids(partitions_final, snid, df_partition, dict_snid_windows, dict_cols):
    aux_snid = copy.copy(snid)
    df_aux = df_partition[df_partition[dict_cols['oid']] == snid]
    num_windows = dict_snid_windows[snid] 
    
    for i in range(num_windows+1):
        df_aux.loc[df_aux[dict_cols['oid']] == aux_snid, dict_cols['oid']] = '{}_{}'.format(snid, i)
        partitions_final.append(df_aux.copy())
        aux_snid = '{}_{}'.format(snid, i)

partitions_final = []

ids_test = test_partition[dict_cols['oid']].values
for snid in ids_test:
    expands_ids(partitions_final, snid, test_partition, df_a[1], dict_cols)

/tmp/ipykernel_1918420/2483919658.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aux.loc[df_aux[dict_cols['oid']] == aux_snid, dict_cols['oid']] = '{}_{}'.format(snid, i)


In [ ]:
partitions_final[0]

,oid,alerceclass,partition
index,,,
23577,ZTF18adldhip_0,AGN,test


In [ ]:
# Merge between light curves and partitions

def ordered_partitions(df_lc, partitions, fold, dict_cols):
    this_partition = partitions[(partitions['partition'] == 'training_%d' %  fold) | \
                                (partitions['partition'] == 'validation_%d' % fold) | \
                                (partitions['partition'] == 'test')]
    this_partition       = this_partition.set_index(dict_cols['oid'])
    this_partition_final = this_partition.filter(items=df_lc.index, axis=0)
    this_partition_final['unique_id'] = np.arange(len(this_partition_final))
    this_partition_final = this_partition_final.reset_index().rename(columns={'index': dict_cols['oid']})
    this_partition_final = this_partition_final.set_index('unique_id')
    return this_partition_final

num_folds = 5
all_partitions = {}
for fold in range(num_folds):
    all_partitions['fold_%s' % fold] = ordered_partitions(df_lcs, 
                                                          df_partitions, 
                                                          fold,
                                                          dict_cols)

In [ ]:
final_windows_merged_df

sigma_flux_diff_ujy   
oid          window_id                                                      
ZTF17aaaaabj 0          [8.440242655407886, 9.70802999032586, 20.01827...  \
             1          [4.335927383739846, 2.337136437970241, 2.61231...   
             2          [3.3180606229102576, 3.4208377390652815, 3.416...   
             3          [4.021680458844159, 4.18601374614786, 4.365849...   
             4          [7.990314418768771, 8.825867966180457, 10.4053...   
...                                                                   ...   
ZTF23aboxuyi 0          [4.789168652528449, 6.6113980531103085, 4.0210...   
             1          [10.31127100662421, 7.095175466434133, 4.76234...   
             2          [8.438503543741724, 9.441484804282839, 5.70083...   
ZTF23abpbdyh 0          [5.791383450140314, 3.544662523295157, 2.92826...   
ZTF23abpbuha 0          [13.62036221217015, 14.48718316429837, 11.4438...   

                                                                 detected   
oid          window_id                                                      
ZTF17aaaaabj 0          [True, True, True, False, True, True, True, Tr...  \
             1          [True, True, True, True, True, True, True, Tru...   
             2          [True, True, True, True, True, True, True, Tru...   
             3          [False, False, False, True, True, True, True, ...   
             4          [True, True, True, True, True, True, True, Tru...   
...                                                                   ...   
ZTF23aboxuyi 0          [False, False, False, False, False, False, Fal...   
             1          [False, False, False, False, False, False, Tru...   
             2          [False, False, False, False, False, False, Fal...   
ZTF23abpbdyh 0          [False, False, False, False, False, False, Fal...   
ZTF23abpbuha 0                      [True, False, True, True, True, True]   

                                                            flux_diff_ujy   
oid          window_id                                                      
ZTF17aaaaabj 0          [-249.57181969652103, 52.815739767359396, -253...  \
             1          [318.88202860232195, -230.68964424237782, -148...   
             2          [-343.69137256798797, -341.49161921565184, -35...   
             3          [19.587287226180354, 4.68433612312114, 0.74238...   
             4          [196.79623824291224, 301.52910950582645, -288....   
...                                                                   ...   
ZTF23aboxuyi 0          [1.1547001816474889, -0.84922994538376, 0.5157...   
             1          [22.27578010361939, 27.833914276248418, 15.723...   
             2          [13.20777222894057, 17.15569121649437, 12.1412...   
ZTF23abpbdyh 0          [6.666708298405723, 3.4420276517688113, 3.3954...   
ZTF23abpbuha 0          [169.48633315872158, 32.28439269439875, 157.16...   

                                                                      fid   
oid          window_id                                                      
ZTF17aaaaabj 0          [1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, ...  \
             1          [2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, ...   
             2          [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...   
             3          [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...   
             4          [1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...   
...                                                                   ...   
ZTF23aboxuyi 0          [1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, ...   
             1          [1, 2, 2, 1, 1, 3, 2, 1, 2, 1, 3, 2, 3, 2, 1, ...   
             2          [2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, ...   
ZTF23abpbdyh 0                             [2, 2, 1, 2, 1, 2, 1, 1, 1, 2]   
ZTF23abpbuha 0                                         [2, 1, 2, 1, 2, 2]   

                                              

In [59]:
df_lc = pp.drop(['alerceclass'], axis=1)
print(df_lc.shape)
df_lc.head()

(253017, 7)


,oid,window_id,detected,mjd,sigma_flux_diff_ujy,flux_diff_ujy,fid
0,ZTF17aaaaabj,0,"[True, True, True, False, True, True, True, Tr...","[58206.16430559987, 58231.14488429995, 58234.1...","[8.440242655407886, 9.70802999032586, 20.01827...","[-249.57181969652103, 52.815739767359396, -253...","[1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, ..."
1,ZTF17aaaaabj,1,"[True, True, True, True, True, True, True, Tru...","[58491.22325230017, 58493.221169000026, 58493....","[4.335927383739846, 2.337136437970241, 2.61231...","[318.88202860232195, -230.68964424237782, -148...","[2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, ..."
2,ZTF17aaaaabj,2,"[True, True, True, True, True, True, True, Tru...","[58806.42067130003, 58806.421134300064, 58806....","[3.3180606229102576, 3.4208377390652815, 3.416...","[-343.69137256798797, -341.49161921565184, -35...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
3,ZTF17aaaaabj,3,"[False, False, False, True, True, True, True, ...","[59043.470949099865, 59043.47599539999, 59043....","[4.021680458844159, 4.18601374614786, 4.365849...","[19.587287226180354, 4.68433612312114, 0.74238...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,ZTF17aaaaabj,4,"[True, True, True, True, True, True, True, Tru...","[59294.158067100216, 59294.19618060021, 59296....","[7.990314418768771, 8.825867966180457, 10.4053...","[196.79623824291224, 301.52910950582645, -288....","[1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


In [60]:
dict_info = {'undersampling_windows': { # Solo funciona con type_windows: windows
            'apply': True,
            'same_windows_by_folds': True,
            'keep_classes': [
                "SNIbc",
                "SNIIn",
                "SLSN",
                "TDE",
                "SNIIb",
                "Microlensing",
            ]
        }
}

name_cols = ['oid', 'window_id', 'alerceclass', 'partition']

test_partition = df_partitions[df_partitions['partition'] == 'test']

partitions_final = []
df_partition_expand = pd.merge(test_partition, df_lc, on='oid', how='outer').dropna()[name_cols]
partitions_final.append(df_partition_expand)


for fold in range(5):
    print('We are expanding the number of windows per light curve in fold {}...'.format(fold))
    this_partitions = df_partitions[(df_partitions['partition'] == 'training_%d' %  fold) | \
                                    (df_partitions['partition'] == 'validation_%d' % fold)]
    
    df_partition_expand = pd.merge(this_partitions, df_lc, on='oid', how='outer').dropna()[name_cols]
    

    if dict_info['undersampling_windows']['apply']:
        if dict_info['undersampling_windows']['same_windows_by_folds']: 
            if fold == 0:
                df_keep_windows = df_partition_expand[
                    df_partition_expand['alerceclass'].isin(dict_info['undersampling_windows']['keep_classes'])
                    ]
                df_partition_expand = df_partition_expand.loc[~df_partition_expand.index.isin(df_keep_windows.index)]

                df_random_window_ids = df_partition_expand.groupby('oid').apply(lambda group: group.sample(n=1)).reset_index(drop=True)
                df_final_windows = pd.concat([df_keep_windows, df_random_window_ids])
                df_aux = df_final_windows.copy()

            else:
                df_final_windows = pd.merge(
                    df_partition_expand, 
                    df_aux[['oid', 'window_id']], 
                    on=['oid', 'window_id']
                )

        else:
            df_keep_windows = df_partition_expand[
                df_partition_expand['alerceclass'].isin(dict_info['undersampling_windows']['keep_classes'])
                ]
            df_partition_expand = df_partition_expand.loc[~df_partition_expand.index.isin(df_keep_windows.index)]

            df_random_window_ids = df_partition_expand.groupby('oid').apply(lambda group: group.sample(n=1)).reset_index(drop=True)
            df_final_windows = pd.concat([df_keep_windows, df_random_window_ids])

        partitions_final.append(df_final_windows)

    else:
        partitions_final.append(df_partition_expand)

We are expanding the number of windows per light curve in fold 0...
We are expanding the number of windows per light curve in fold 1...
We are expanding the number of windows per light curve in fold 2...
We are expanding the number of windows per light curve in fold 3...
We are expanding the number of windows per light curve in fold 4...


In [61]:
df_partitions = pd.concat(partitions_final)
df_partitions

,oid,window_id,alerceclass,partition
0,ZTF18adldhip,0,AGN,test
1,ZTF18adldhip,1,AGN,test
2,ZTF18adldhip,2,AGN,test
3,ZTF18adldhip,3,AGN,test
4,ZTF19aasbgeb,0,QSO,test
...,...,...,...,...
35972,ZTF19acokmuo,7,AGN,validation_4
35973,ZTF18abxoxih,1,Blazar,validation_4
35974,ZTF18adbhpsf,1,Blazar,validation_4
35975,ZTF21aamkwra,0,Periodic-Other,validation_4


In [62]:
df_partitions['oid'] = df_partitions['oid'].astype(str) + '_' + df_partitions['window_id'].astype(str)
df_partitions = df_partitions.drop(columns=['window_id'])
df_partitions

,oid,alerceclass,partition
0,ZTF18adldhip_0,AGN,test
1,ZTF18adldhip_1,AGN,test
2,ZTF18adldhip_2,AGN,test
3,ZTF18adldhip_3,AGN,test
4,ZTF19aasbgeb_0,QSO,test
...,...,...,...
35972,ZTF19acokmuo_7,AGN,validation_4
35973,ZTF18abxoxih_1,Blazar,validation_4
35974,ZTF18adbhpsf_1,Blazar,validation_4
35975,ZTF21aamkwra_0,Periodic-Other,validation_4


In [63]:
df_lc['oid'] = df_lc['oid'].astype(str) + '_' + df_lc['window_id'].astype(str)
df_lc = df_lc.drop(columns=['window_id'])
df_lc

,oid,detected,mjd,sigma_flux_diff_ujy,flux_diff_ujy,fid
0,ZTF17aaaaabj_0,"[True, True, True, False, True, True, True, Tr...","[58206.16430559987, 58231.14488429995, 58234.1...","[8.440242655407886, 9.70802999032586, 20.01827...","[-249.57181969652103, 52.815739767359396, -253...","[1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, ..."
1,ZTF17aaaaabj_1,"[True, True, True, True, True, True, True, Tru...","[58491.22325230017, 58493.221169000026, 58493....","[4.335927383739846, 2.337136437970241, 2.61231...","[318.88202860232195, -230.68964424237782, -148...","[2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, ..."
2,ZTF17aaaaabj_2,"[True, True, True, True, True, True, True, Tru...","[58806.42067130003, 58806.421134300064, 58806....","[3.3180606229102576, 3.4208377390652815, 3.416...","[-343.69137256798797, -341.49161921565184, -35...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
3,ZTF17aaaaabj_3,"[False, False, False, True, True, True, True, ...","[59043.470949099865, 59043.47599539999, 59043....","[4.021680458844159, 4.18601374614786, 4.365849...","[19.587287226180354, 4.68433612312114, 0.74238...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,ZTF17aaaaabj_4,"[True, True, True, True, True, True, True, Tru...","[59294.158067100216, 59294.19618060021, 59296....","[7.990314418768771, 8.825867966180457, 10.4053...","[196.79623824291224, 301.52910950582645, -288....","[1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
...,...,...,...,...,...,...
253012,ZTF23aboxuyi_0,"[False, False, False, False, False, False, Fal...","[58872.165138899814, 58875.09256940009, 58875....","[4.789168652528449, 6.6113980531103085, 4.0210...","[1.1547001816474889, -0.84922994538376, 0.5157...","[1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, ..."
253013,ZTF23aboxuyi_1,"[False, False, False, False, False, False, Tru...","[59418.46142360009, 59422.43584490009, 59426.4...","[10.31127100662421, 7.095175466434133, 4.76234...","[22.27578010361939, 27.833914276248418, 15.723...","[1, 2, 2, 1, 1, 3, 2, 1, 2, 1, 3, 2, 3, 2, 1, ..."
253014,ZTF23aboxuyi_2,"[False, False, False, False, False, False, Fal...","[59803.41695600003, 59803.45730319992, 59808.4...","[8.438503543741724, 9.441484804282839, 5.70083...","[13.20777222894057, 17.15569121649437, 12.1412...","[2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, ..."
253015,ZTF23abpbdyh_0,"[False, False, False, False, False, False, Fal...","[60236.48218750022, 60238.51758099999, 60242.4...","[5.791383450140314, 3.544662523295157, 2.92826...","[6.666708298405723, 3.4420276517688113, 3.3954...","[2, 2, 1, 2, 1, 2, 1, 1, 1, 2]"


In [46]:
df_partitions.oid.unique()

array(['ZTF18adldhip_0', 'ZTF18adldhip_1', 'ZTF18adldhip_2', ...,
       'ZTF23aboxuyi_1', 'ZTF23abpbdyh_0', 'ZTF23abpbuha_0'], dtype=object)

In [47]:
df_lc[df_lc.oid.isin(df_partitions.oid.unique())]

,oid,detected,mjd,sigma_flux_diff_ujy,flux_diff_ujy,fid
3,ZTF17aaaaabj_3,"[False, False, False, True, True, True, True, ...","[59043.470949099865, 59043.47599539999, 59043....","[4.021680458844159, 4.18601374614786, 4.365849...","[19.587287226180354, 4.68433612312114, 0.74238...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
12,ZTF17aaaaaej_5,"[True, True, True, True, True, True, True, Tru...","[59787.484143500216, 59787.48666669987, 59788....","[4.011610460702935, 3.8685977854458247, 4.1426...","[132.73786205442275, 130.65187755858432, -159....","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, ..."
14,ZTF17aaaaafz_0,"[False, True, False, True, False, False, False...","[58206.16430559987, 58231.14488429995, 58234.1...","[8.334198519710379, 9.206879583386137, 17.2728...","[20.93581016164701, 85.15186635623513, -14.291...","[1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, ..."
15,ZTF17aaaaafz_1,"[True, True, True, True, True, True, True, Tru...","[58503.22148150019, 58505.11980320001, 58505.1...","[10.66184021489383, 6.68725946520036, 9.476142...","[-82.19465034260726, -301.0504183933542, -73.0...","[1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, ..."
16,ZTF17aaaaafz_2,"[True, True, True, True, True, True, True, Tru...","[58806.42609950015, 58806.4265509001, 58806.42...","[4.830327496378002, 4.950453165721698, 4.83388...","[198.88405845006156, 193.70281097080766, 190.3...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
...,...,...,...,...,...,...
253010,ZTF23abobwsd_0,"[False, False, False, False, False, False, Fal...","[60221.32445599977, 60221.36464119982, 60223.2...","[4.50945021489488, 4.5113689816309765, 4.14554...","[-6.1656230209345555, 1.9790447084421814, 3.05...","[2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, ..."
253011,ZTF23abofpay_0,"[False, False, False, False, False, False, Fal...","[60221.207939799875, 60221.24505789997, 60223....","[4.365858020242146, 5.083975012448924, 4.79596...","[-0.18967056540238775, -3.1798984307751734, -4...","[1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, ..."
253013,ZTF23aboxuyi_1,"[False, False, False, False, False, False, Tru...","[59418.46142360009, 59422.43584490009, 59426.4...","[10.31127100662421, 7.095175466434133, 4.76234...","[22.27578010361939, 27.833914276248418, 15.723...","[1, 2, 2, 1, 1, 3, 2, 1, 2, 1, 3, 2, 3, 2, 1, ..."
253015,ZTF23abpbdyh_0,"[False, False, False, False, False, False, Fal...","[60236.48218750022, 60238.51758099999, 60242.4...","[5.791383450140314, 3.544662523295157, 2.92826...","[6.666708298405723, 3.4420276517688113, 3.3954...","[2, 2, 1, 2, 1, 2, 1, 1, 1, 2]"


In [65]:
from data.src.partitions import get_partitions, ordered_partitions

In [68]:
def ordered_partitions(df_lc, partitions, fold, dict_cols):
    this_partition = partitions[(partitions['partition'] == 'training_%d' %  fold) | \
                                (partitions['partition'] == 'validation_%d' % fold) | \
                                (partitions['partition'] == 'test')]
    print(this_partition)
    this_partition       = this_partition.set_index(dict_cols['oid'])
    print(this_partition)
    this_partition_final = this_partition.filter(items=df_lc.index, axis=0)
    this_partition_final['unique_id'] = np.arange(len(this_partition_final))
    this_partition_final = this_partition_final.reset_index().rename(columns={'index': dict_cols['oid']})
    this_partition_final = this_partition_final.set_index('unique_id')
    return this_partition_final

In [69]:
num_folds = 5
all_partitions = {}
for fold in range(num_folds):
    all_partitions['fold_%s' % fold] = ordered_partitions(df_lc, 
                                                          df_partitions, 
                                                          fold,
                                                          dict_cols)

                  oid alerceclass     partition
0      ZTF18adldhip_0         AGN          test
1      ZTF18adldhip_1         AGN          test
2      ZTF18adldhip_2         AGN          test
3      ZTF18adldhip_3         AGN          test
4      ZTF19aasbgeb_0         QSO          test
...               ...         ...           ...
32328  ZTF23abnzxou_2       RSCVn    training_0
32329  ZTF23abobwsd_0        SNII    training_0
32330  ZTF23aboxuyi_0         QSO  validation_0
32331  ZTF23abpbdyh_0     CV/Nova  validation_0
32332  ZTF23abpbuha_0        SNII    training_0

[86478 rows x 3 columns]
               alerceclass     partition
oid                                     
ZTF18adldhip_0         AGN          test
ZTF18adldhip_1         AGN          test
ZTF18adldhip_2         AGN          test
ZTF18adldhip_3         AGN          test
ZTF19aasbgeb_0         QSO          test
...                    ...           ...
ZTF23abnzxou_2       RSCVn    training_0
ZTF23abobwsd_0        SNII   

In [67]:
all_partitions

{'fold_0': Empty DataFrame
 Columns: [oid, alerceclass, partition]
 Index: [],
 'fold_1': Empty DataFrame
 Columns: [oid, alerceclass, partition]
 Index: [],
 'fold_2': Empty DataFrame
 Columns: [oid, alerceclass, partition]
 Index: [],
 'fold_3': Empty DataFrame
 Columns: [oid, alerceclass, partition]
 Index: [],
 'fold_4': Empty DataFrame
 Columns: [oid, alerceclass, partition]
 Index: []}

In [17]:
partitions_final[2].sort_values(by='oid').oid.nunique()

33382

In [18]:
partitions_final[3].sort_values(by='oid').oid.nunique()

33382

In [19]:
partitions_final[1].sort_values(by='oid')

,oid,window_id,alerceclass,partition
0,ZTF17aaaaabj,3,RRLc,training_0
1,ZTF17aaaaaej,5,RRLc,validation_0
2,ZTF17aaaaajg,1,RRLc,training_0
3,ZTF17aaaaavq,4,LPV,validation_0
4,ZTF17aaaaaya,6,RRLab,training_0
...,...,...,...,...
32329,ZTF23abobwsd,0,SNII,training_0
6445,ZTF23abofpay,0,SNIbc,training_0
32330,ZTF23aboxuyi,1,QSO,validation_0
32331,ZTF23abpbdyh,0,CV/Nova,validation_0


In [20]:
partitions_final[2].sort_values(by='oid')

,oid,window_id,alerceclass,partition
2117,ZTF17aaaaabj,3,RRLc,training_1
18375,ZTF17aaaaaej,5,RRLc,training_1
2758,ZTF17aaaaajg,1,RRLc,training_1
12336,ZTF17aaaaavq,4,LPV,training_1
24433,ZTF17aaaaaya,6,RRLab,training_1
...,...,...,...,...
31187,ZTF23abobwsd,0,SNII,validation_1
1125,ZTF23abofpay,0,SNIbc,training_1
21240,ZTF23aboxuyi,1,QSO,training_1
18230,ZTF23abpbdyh,0,CV/Nova,training_1


In [21]:
partitions_final[2].groupby('alerceclass').count().sort_values(by='oid')

,oid,window_id,partition
alerceclass,,,
Microlensing,125,125,125
SNIIb,317,317,317
TDE,345,345,345
SLSN,490,490,490
SNIIn,855,855,855
Periodic-Other,1067,1067,1067
Blazar,1182,1182,1182
SNII,1289,1289,1289
DSCT,1326,1326,1326


In [22]:
partitions_final[2].groupby('alerceclass').count().sort_values(by='oid')

,oid,window_id,partition
alerceclass,,,
Microlensing,125,125,125
SNIIb,317,317,317
TDE,345,345,345
SLSN,490,490,490
SNIIn,855,855,855
Periodic-Other,1067,1067,1067
Blazar,1182,1182,1182
SNII,1289,1289,1289
DSCT,1326,1326,1326


In [25]:
df_aux = pd.concat([partitions_final[0], partitions_final[1]])
df_aux

,oid,window_id,alerceclass,partition
0,ZTF18adldhip,0,AGN,test
1,ZTF18adldhip,1,AGN,test
2,ZTF18adldhip,2,AGN,test
3,ZTF18adldhip,3,AGN,test
4,ZTF19aasbgeb,0,QSO,test
...,...,...,...,...
32328,ZTF23abnzxou,1,RSCVn,training_0
32329,ZTF23abobwsd,0,SNII,training_0
32330,ZTF23aboxuyi,1,QSO,validation_0
32331,ZTF23abpbdyh,0,CV/Nova,validation_0


In [23]:
df_lc

,oid,window_id,detected,mjd,sigma_flux_diff_ujy,flux_diff_ujy,fid
0,ZTF17aaaaabj,0,"[True, True, True, False, True, True, True, Tr...","[58206.16430559987, 58231.14488429995, 58234.1...","[8.440242655407886, 9.70802999032586, 20.01827...","[-249.57181969652103, 52.815739767359396, -253...","[1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, ..."
1,ZTF17aaaaabj,1,"[True, True, True, True, True, True, True, Tru...","[58491.22325230017, 58493.221169000026, 58493....","[4.335927383739846, 2.337136437970241, 2.61231...","[318.88202860232195, -230.68964424237782, -148...","[2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, ..."
2,ZTF17aaaaabj,2,"[True, True, True, True, True, True, True, Tru...","[58806.42067130003, 58806.421134300064, 58806....","[3.3180606229102576, 3.4208377390652815, 3.416...","[-343.69137256798797, -341.49161921565184, -35...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
3,ZTF17aaaaabj,3,"[False, False, False, True, True, True, True, ...","[59043.470949099865, 59043.47599539999, 59043....","[4.021680458844159, 4.18601374614786, 4.365849...","[19.587287226180354, 4.68433612312114, 0.74238...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,ZTF17aaaaabj,4,"[True, True, True, True, True, True, True, Tru...","[59294.158067100216, 59294.19618060021, 59296....","[7.990314418768771, 8.825867966180457, 10.4053...","[196.79623824291224, 301.52910950582645, -288....","[1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
...,...,...,...,...,...,...,...
253012,ZTF23aboxuyi,0,"[False, False, False, False, False, False, Fal...","[58872.165138899814, 58875.09256940009, 58875....","[4.789168652528449, 6.6113980531103085, 4.0210...","[1.1547001816474889, -0.84922994538376, 0.5157...","[1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, ..."
253013,ZTF23aboxuyi,1,"[False, False, False, False, False, False, Tru...","[59418.46142360009, 59422.43584490009, 59426.4...","[10.31127100662421, 7.095175466434133, 4.76234...","[22.27578010361939, 27.833914276248418, 15.723...","[1, 2, 2, 1, 1, 3, 2, 1, 2, 1, 3, 2, 3, 2, 1, ..."
253014,ZTF23aboxuyi,2,"[False, False, False, False, False, False, Fal...","[59803.41695600003, 59803.45730319992, 59808.4...","[8.438503543741724, 9.441484804282839, 5.70083...","[13.20777222894057, 17.15569121649437, 12.1412...","[2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, ..."
253015,ZTF23abpbdyh,0,"[False, False, False, False, False, False, Fal...","[60236.48218750022, 60238.51758099999, 60242.4...","[5.791383450140314, 3.544662523295157, 2.92826...","[6.666708298405723, 3.4420276517688113, 3.3954...","[2, 2, 1, 2, 1, 2, 1, 1, 1, 2]"
